In [186]:
import pandas as pd
import numpy as np
import math
import datetime

In [2]:
#---------------------------------------------------------#
# Prototyping
#---------------------------------------------------------#

In [144]:
eds_cols = {}
window = 7

In [145]:
# file location for the csv file
file = "./testing_data.csv"
# return pandas dataframe of the csv file
df = pd.read_csv(file)
# remove all NaN entries
df.drop(df.filter(regex="Unname"),axis=1, inplace=True)
a=df.iterrows()
i = 0
for index,row in a:
    if math.isnan(row[2]):
        break
    else:
        i = i + 1
table = df[0:i]
table.head(10)

,Date,Time,Temperature(C),Humidity(%),GPOA(W/M2),EDS(#),OCV_Before(V),OCV_After(V),SCC_Before(A),SCC_After(A),CTRL1_OCV(V),CTRL1_SCC(A),CTRL2_OCV(V),CTRL2_SCC(A),EDS_PWR_Before(W),EDS_PWR_After(W),CTRL1_PWR(W),CTRL2_PWR(W)
0,1/22/2020,12:33:26,32.4,28.3,-1,1,0.307863,0.307863,0.030797,0.030797,19.826405,0.066726,0.307863,0.030797,0.004043,0.004043,1.280990,0.004043
1,1/22/2020,12:34:51,32.4,28.0,-1,2,0.307863,0.369436,0.025664,0.030797,19.887978,0.066726,0.307863,0.030797,0.003369,0.005342,1.285077,0.004043
2,1/22/2020,12:37:11,31.3,27.4,-1,3,17.609789,17.609789,0.041062,0.035930,19.887978,0.066726,0.307863,0.030797,0.698502,0.611189,1.286275,0.004120
3,1/22/2020,12:38:35,31.3,45.5,-1,4,0.307863,0.307863,0.030797,0.030797,19.949551,0.066726,0.307863,0.030797,0.004120,0.004120,1.290363,0.004120
4,1/22/2020,12:40:58,31.0,29.9,-1,5,0.307863,0.307863,0.030797,0.030797,19.887978,0.066726,0.246291,0.030797,0.004141,0.004141,1.286603,0.002925
5,1/23/2020,12:33:26,32.4,28.3,-1,1,0.307863,0.307863,0.030797,0.030797,19.826405,0.066726,0.307863,0.030797,0.004043,0.004043,1.280990,0.004043
6,1/23/2020,12:34:51,32.4,28.0,-1,2,0.307863,0.369436,0.025664,0.030797,19.887978,0.066726,0.307863,0.030797,0.003369,0.005342,1.285077,0.004043
7,1/23/2020,12:37:11,31.3,27.4,-1,3,17.609789,17.609789,0.041062,0.035930,19.887978,0.066726,0.307863,0.030797,0.698502,0.611189,1.286275,0.004120
8,1/23/2020,12:38:35,31.3,45.5,-1,4,0.307863,0.307863,0.030797,0.030797,19.949551,0.066726,0.307863,0.030797,0.004120,0.004120,1.290363,0.004120
9,1/23/2020,12:40:58,31.0,29.9,-1,5,0.307863,0.307863,0.030797,0.030797,19.887978,0.066726,0.246291,0.030797,0.004141,0.004141,1.286603,0.002925


In [146]:
# define dictionary for new table

In [160]:
eds_cols = {}
window = 7

In [161]:
# sort date based on time window

In [162]:
col_name = 'Date'
counter = 0
date_string = ''
new_col = {col_name:[]}

In [163]:
for x in df[col_name]:
    if counter == 0:
        # store the first date to date_string
        date_string = date_string + x + '-'
        # increase counter
        counter = counter + 1
    elif counter == ((window*5)-1):
        # store the last date
        date_string = date_string + x
        # store to dictionary for dataframe for pre and post entry
        for x in range (5):
            new_col[col_name].append(date_string)
        # reset date_string
        date_string = ''
        # reset counter
        counter = 0
    else:
        # increase counter
        counter = counter + 1
eds_cols.update(new_col)   

In [164]:
#eds_cols

In [165]:
# solar noon time sorting

In [166]:
col_name = 'Time'
counter = 0
index_counter = 0
time_data = [0,0,0,0,0]
new_col = {col_name:[]}

In [167]:
for x in df[col_name]:
    # convert time to seconds
    a,b,c = x.split(':')
    hour = int(a)*60*60
    minute = int(b)*60
    second = int(c)
    total_sec = hour + minute + second
    if counter == ((window*5)-1):
        # check if its pre or post date points
        time_data[index_counter] = time_data[index_counter] + total_sec
        # take average seconds for pre data
        avg_time_data = []
        for x in time_data:
            avg_time_data.append(x/window)
        # convert seconds back to time string using datetime for pre data
        time = []
        for x in avg_time_data:
            time.append(str(datetime.timedelta(seconds = int(x))))
        # append to the dictionary
        new_col[col_name] = new_col[col_name]+ time
        # reset counter
        counter = 0
        index_counter = 0
        time_data = [0,0,0,0,0]
    else:
        if index_counter == 5:
            index_counter = 0
        # check if its pre or post date points
        time_data[index_counter] = time_data[index_counter] + total_sec
        # increase counters
        counter = counter + 1
        index_counter = index_counter + 1
eds_cols.update(new_col)

In [168]:
eds_cols

{'Date': ['1/22/2020-1/28/2020',
  '1/22/2020-1/28/2020',
  '1/22/2020-1/28/2020',
  '1/22/2020-1/28/2020',
  '1/22/2020-1/28/2020'],
 'Time': ['12:33:26', '12:34:51', '12:37:11', '12:38:35', '12:40:58']}

In [169]:
# numerical data sorting

In [170]:
col_name = 'Temperature(C)'
counter = 0
index_counter = 0
data = [0,0,0,0,0]
new_col = {col_name:[]}

In [171]:
for x in df[col_name]:
    if counter == ((window*5)-1):
        # modify nth time data
        data[index_counter] = data[index_counter] + x
        # take average seconds for data
        avg_data = []
        for i in data:
            avg_data.append(i/window)
        # append to the dictionary
        new_col[col_name] = new_col[col_name]+ avg_data
        # reset counter
        index_counter = 0
        counter = 0
        data = [0,0,0,0,0]
    else:
        if index_counter == 5:
            index_counter = 0
        # check if its pre or post date points
        data[index_counter] = data[index_counter] + x
        # increase counters
        counter = counter + 1
        index_counter = index_counter + 1
eds_cols.update(new_col)

In [172]:
eds_cols

{'Date': ['1/22/2020-1/28/2020',
  '1/22/2020-1/28/2020',
  '1/22/2020-1/28/2020',
  '1/22/2020-1/28/2020',
  '1/22/2020-1/28/2020'],
 'Time': ['12:33:26', '12:34:51', '12:37:11', '12:38:35', '12:40:58'],
 'Temperature(C)': [32.4, 32.4, 31.300000000000004, 31.300000000000004, 31.0]}

In [173]:
# pre/post label sorting, 

In [174]:
col_name2 = 'EDS(#)'
counter = 0
new_col2= {col_name2:[]}

In [175]:
for x in df[col_name2]:
    if counter == ((window*5)-1):
        # append EDS number
        for x in range(5):
            new_col2[col_name2].append("EDS"+str(x+1))
        # reset counter
        counter = 0
    else:
        # increase counter
        counter = counter + 1
eds_cols.update(new_col2)
#eds_cols

In [176]:
eds_cols

{'Date': ['1/22/2020-1/28/2020',
  '1/22/2020-1/28/2020',
  '1/22/2020-1/28/2020',
  '1/22/2020-1/28/2020',
  '1/22/2020-1/28/2020'],
 'Time': ['12:33:26', '12:34:51', '12:37:11', '12:38:35', '12:40:58'],
 'Temperature(C)': [32.4, 32.4, 31.300000000000004, 31.300000000000004, 31.0],
 'EDS(#)': ['EDS1', 'EDS2', 'EDS3', 'EDS4', 'EDS5']}

In [12]:
#---------------------------------------------------------#
# Transform to functions to use
#---------------------------------------------------------#

In [20]:
cols_list = ['Temperature(C)', 'Humidity(%)', 'GPOA(W/M2)', 'OCV_Before(V)', 'OCV_After(V)', 'SCC_Before(A)', 'SCC_After(A)', 'CTRL1_OCV(V)', 'CTRL1_SCC(A)', 'CTRL2_OCV(V)', 'CTRL2_SCC(A)', 'EDS_PWR_Before(W)', 'EDS_PWR_After(W)', 'CTRL1_PWR(W)', 'CTRL2_PWR(W)']

In [188]:
def read_data(name):
    # file location for the csv file
    file = name
    # return pandas dataframe of the csv file
    df = pd.read_csv(file)
    # remove all NaN entries
    df.drop(df.filter(regex="Unname"),axis=1, inplace=True)
    a=df.iterrows()
    i = 0
    for index,row in a:
        if math.isnan(row[2]):
            break
        else:
            i = i + 1
    table = df[0:i]
    
    return table

In [189]:
def sort_dates(eds_cols, window):
    # read the noon_data csv file
    df = read_data("./testing_data.csv")
    # declare initial variables
    col_name = 'Date'
    counter = 0
    date_string = ''
    new_col = {col_name:[]}
    # go through the Date column of the noon data csv file
    for x in df[col_name]:
        if counter == 0:
            # store the first date to date_string
            date_string = date_string + x + '-'
            # increase counter
            counter = counter + 1
        elif counter == ((window*5)-1):
            # store the last date
            date_string = date_string + x
            # store to dictionary for dataframe for pre and post entry
            for x in range (5):
                new_col[col_name].append(date_string)
            # reset date_string
            date_string = ''
            # reset counter
            counter = 0
        else:
            # increase counter
            counter = counter + 1
    eds_cols.update(new_col)    
    return eds_cols

In [190]:
def sort_time(eds_cols, window):
    # read the noon_data csv file
    df = read_data("./testing_data.csv")
    # declare initial variables
    col_name = 'Time'
    counter = 0
    index_counter = 0
    time_data = [0,0,0,0,0]
    new_col = {col_name:[]}
    # go through the Time column of the noon data csv file
    for x in df[col_name]:
        # convert time to seconds
        a,b,c = x.split(':')
        hour = int(a)*60*60
        minute = int(b)*60
        second = int(c)
        total_sec = hour + minute + second
        if counter == ((window*5)-1):
            # check if its pre or post date points
            time_data[index_counter] = time_data[index_counter] + total_sec
            # take average seconds for pre data
            avg_time_data = []
            for x in time_data:
                avg_time_data.append(x/window)
            # convert seconds back to time string using datetime for pre data
            time = []
            for x in avg_time_data:
                time.append(str(datetime.timedelta(seconds = int(x))))
            # append to the dictionary
            new_col[col_name] = new_col[col_name]+ time
            # reset counter
            counter = 0
            index_counter = 0
            time_data = [0,0,0,0,0]
        else:
            if index_counter == 5:
                index_counter = 0
            # check if its pre or post date points
            time_data[index_counter] = time_data[index_counter] + total_sec
            # increase counters
            counter = counter + 1
            index_counter = index_counter + 1
    eds_cols.update(new_col)
    return eds_cols

In [191]:
def sort_labels(eds_cols, window):
    # read the noon_data csv file
    df = read_data("./testing_data.csv")
    # declare initial variables
    col_name2 = 'EDS(#)'
    counter = 0
    new_col2= {col_name2:[]}
    # go through the labels column of the noon data csv file
    for x in df[col_name2]:
        if counter == ((window*5)-1):
            # append EDS number
            for x in range(5):
                new_col2[col_name2].append("EDS"+str(x+1))
            # reset counter
            counter = 0
        else:
            # increase counter
            counter = counter + 1
    eds_cols.update(new_col2)
    return eds_cols

In [192]:
def sort_data(name, eds_cols, window):
    # read the noon_data csv file
    df = read_data("./testing_data.csv")
    # declare initial variables
    col_name = name #'Temperature(C)'
    counter = 0
    index_counter = 0
    data = [0,0,0,0,0]
    new_col = {col_name:[]}
    # go through the measurements data columns of the noon data csv file
    for x in df[col_name]:
        if counter == ((window*5)-1):
            # modify nth time data
            data[index_counter] = data[index_counter] + x
            # take average seconds for data
            avg_data = []
            for i in data:
                avg_data.append(i/window)
            # append to the dictionary
            new_col[col_name] = new_col[col_name]+ avg_data
            # reset counter
            index_counter = 0
            counter = 0
            data = [0,0,0,0,0]
        else:
            if index_counter == 5:
                index_counter = 0
            # check if its pre or post date points
            data[index_counter] = data[index_counter] + x
            # increase counters
            counter = counter + 1
            index_counter = index_counter + 1
    eds_cols.update(new_col)
    return eds_cols

In [193]:
def get_avg_testing_data(cols_list, window):
    # read the noon_data csv file
    df = read_data("./testing_data.csv")
    # declare new dictionary for avg data
    eds_cols = {}
    # sort the date
    eds_cols = sort_dates(eds_cols, window)
    # sort the time
    eds_cols = sort_time(eds_cols, window)
    # sort the pre/post, EDS number
    eds_cols = sort_labels(eds_cols, window)
    # sort all the numerical data
    for x in cols_list:
        eds_cols = sort_data(x, eds_cols, window)
    # create new dataframe
    eds_df = pd.DataFrame(eds_cols)
    return eds_df

In [195]:
# declare metrics for noon data table
#cols_list = ['Temperature(C)', 'Humidity(%)', 'GPOA(W/M2)', 'OCV(V)', 'SCC(A)', 'Power(W)', 'PR', 'SR']
cols_list = ['Temperature(C)', 'Humidity(%)', 'GPOA(W/M2)', 'OCV_Before(V)', 'OCV_After(V)', 'SCC_Before(A)', 'SCC_After(A)', 'CTRL1_OCV(V)', 'CTRL1_SCC(A)', 'CTRL2_OCV(V)', 'CTRL2_SCC(A)', 'EDS_PWR_Before(W)', 'EDS_PWR_After(W)', 'CTRL1_PWR(W)', 'CTRL2_PWR(W)']
# call the function
eds = get_avg_testing_data(cols_list, 3)
eds

,Date,Time,EDS(#),Temperature(C),Humidity(%),GPOA(W/M2),OCV_Before(V),OCV_After(V),SCC_Before(A),SCC_After(A),CTRL1_OCV(V),CTRL1_SCC(A),CTRL2_OCV(V),CTRL2_SCC(A),EDS_PWR_Before(W),EDS_PWR_After(W),CTRL1_PWR(W),CTRL2_PWR(W)
0,1/22/2020-1/24/2020,12:33:26,EDS1,32.4,28.3,-1.0,0.307863,0.307863,0.030797,0.030797,19.826405,0.066726,0.307863,0.030797,0.004043,0.004043,1.280990,0.004043
1,1/22/2020-1/24/2020,12:34:51,EDS2,32.4,28.0,-1.0,0.307863,0.369436,0.025664,0.030797,19.887978,0.066726,0.307863,0.030797,0.003369,0.005342,1.285077,0.004043
2,1/22/2020-1/24/2020,12:37:11,EDS3,31.3,27.4,-1.0,17.609789,17.609789,0.041062,0.035930,19.887978,0.066726,0.307863,0.030797,0.698502,0.611189,1.286275,0.004120
3,1/22/2020-1/24/2020,12:38:35,EDS4,31.3,45.5,-1.0,0.307863,0.307863,0.030797,0.030797,19.949551,0.066726,0.307863,0.030797,0.004120,0.004120,1.290363,0.004120
4,1/22/2020-1/24/2020,12:40:58,EDS5,31.0,29.9,-1.0,0.307863,0.307863,0.030797,0.030797,19.887978,0.066726,0.246291,0.030797,0.004141,0.004141,1.286603,0.002925
5,1/25/2020-1/27/2020,12:33:26,EDS1,32.4,28.3,-1.0,0.307863,0.307863,0.030797,0.030797,19.826405,0.066726,0.307863,0.030797,0.004043,0.004043,1.280990,0.004043
6,1/25/2020-1/27/2020,12:34:51,EDS2,32.4,28.0,-1.0,0.307863,0.369436,0.025664,0.030797,19.887978,0.066726,0.307863,0.030797,0.003369,0.005342,1.285077,0.004043
7,1/25/2020-1/27/2020,12:37:11,EDS3,31.3,27.4,-1.0,17.609789,17.609789,0.041062,0.035930,19.887978,0.066726,0.307863,0.030797,0.698502,0.611189,1.286275,0.004120
8,1/25/2020-1/27/2020,12:38:35,EDS4,31.3,45.5,-1.0,0.307863,0.307863,0.030797,0.030797,19.949551,0.066726,0.307863,0.030797,0.004120,0.004120,1.290363,0.004120
9,1/25/2020-1/27/2020,12:40:58,EDS5,31.0,29.9,-1.0,0.307863,0.307863,0.030797,0.030797,19.887978,0.066726,0.246291,0.030797,0.004141,0.004141,1.286603,0.002925
